# Implementation Alpha 

## Extract Fetal ECG - Final Project 29 

### Kabir Chadha, Maxwell Kretschmer, Jack Rickman



# 1. Unzipping .csv files containing data for the 4 channels over 60 seconds every millisecond

In [ ]:
# DATA UNPACKING
# Load in the data, then uncomment and run:
# !unzip "/content/set-a-text.zip"
# !unzip "/content/set-a-text-test.zip"
# !unzip "/content/set-a-ext-text.zip"
# !unzip "/content/set-a-ext-text-test.zip"

# 2. Grabbing a datatframe of the data from each file, and appending it as a possible sample to select from in the all_records dataframe

In [ ]:
# DATA LOADING AND CLEANING
import pandas as pd
all_records = []

number_of_files = 9

for i in range(1, number_of_files+1):
    df = pd.read_csv("a0{}.csv".format(i), skiprows=[1])
    df.rename(columns={"\'Elapsed time\'": "elapsedTime", "\'AECG1\'": "AECG1", "\'AECG2\'": "AECG2", "\'AECG3\'": "AECG3", "\'AECG4\'":"AECG4"}, inplace=True)
    all_records.append(df)

number_of_files = 76

for i in range(10, number_of_files):
    df = pd.read_csv("a{}.csv".format(i), skiprows=[1])
    df.rename(columns={"\'Elapsed time\'": "elapsedTime", "\'AECG1\'": "AECG1", "\'AECG2\'": "AECG2", "\'AECG3\'": "AECG3", "\'AECG4\'":"AECG4"}, inplace=True)
    all_records.append(df)

# 3. Overview of data with each Channel shown concatenated vertically

In [ ]:
# VISUALIZATION 1
import altair as alt
x = 58 # a59
alt.data_transformers.disable_max_rows()
AECG1 = alt.Chart(all_records[x]).mark_line().encode(
    x=alt.X('elapsedTime:Q', title="Time Elapsed (second)"),
    y='AECG1:Q'
).properties(
    width=2000,
    height=250,
    title = "Overview of Data with each Channel"
)
AECG2 = alt.Chart(all_records[x]).mark_line().encode(
    x=alt.X('elapsedTime:Q', title="Time Elapsed (second)"),
    y='AECG2'
).properties(
    width=2000,
    height=250
)

AECG3 = alt.Chart(all_records[x]).mark_line().encode(
    x=alt.X('elapsedTime:Q', title="Time Elapsed (second)"),
    y='AECG3'
).properties(
    width=2000,
    height=250
)

AECG4 = alt.Chart(all_records[x]).mark_line().encode(
    x=alt.X('elapsedTime:Q', title="Time Elapsed (second)"),
    y='AECG4'
).properties(
    width=2000,
    height=250
)

AECG1 & AECG2 & AECG3 & AECG4

Output hidden; open in https://colab.research.google.com to view.

# 4. Adding extent binding in order to show a zoomed in view of the graph above a smaller view of the whole data

In [ ]:
# VISUALIZATION 2
interval = alt.selection_interval(encodings=['x'])

sliced = all_records[x][:30000]

base = alt.Chart(sliced).mark_line().encode(
    x=alt.X('elapsedTime:Q', title="Time Elapsed (second)"),
    y='AECG1:Q'
)

FullExtent = base.add_selection(
    interval
).properties(
    width=2000,
    height=100
)

ZoomedExtent = base.encode(
    x=alt.X('elapsedTime:Q', scale=alt.Scale(domain=interval.ref()))
).properties(
    width=2000,
    height=500,
    title="Using Extent Binding to Zoom"
)

ZoomedExtent & FullExtent

Output hidden; open in https://colab.research.google.com to view.

# 5. Melting data in order to have a channel column instead of 4 columns for each channel, in order to allow user selection of channel in the visualization.

# 6. Final Implementation of the first main visualization with both extent binding and channel selection interaction

## TODO:
~~ADD FIXED Y AXIS~~

Multi-selection and at top

fix legend
 
 SAVE AS HTML


In [ ]:
sliced_long =  pd.melt(sliced, ['elapsedTime'])

sliced_long.rename(columns = {'variable':'channel', 'value':'ECG'}, inplace = True)
channels = [None] + list(sliced_long['channel'].unique())
channels_labels = ["All"] + list(sliced_long['channel'].unique())
sliced_long.head()

,elapsedTime,channel,ECG
0,0.000,AECG1,-209.0
1,0.001,AECG1,-206.5
2,0.002,AECG1,-202.0
3,0.003,AECG1,-197.0
4,0.004,AECG1,-191.0


In [ ]:
channel_scale = alt.Scale(domain=channels,
                         range=['#FF0000', '#00FF00', '#0099ff', '#4800ff'])

interval = alt.selection_interval(encodings=['x'])

channel_select_box = alt.binding_select(options=channels, labels=channels_labels, name="Channel")
channel_selector = alt.selection_single(fields=['channel'], bind=channel_select_box)

base = alt.Chart(sliced_long).mark_line().encode(
    x=alt.X('elapsedTime:Q', title="Time Elapsed (second)"),
    y=alt.Y('ECG:Q', title="ECG Value", scale=alt.Scale(domain=[min(list(sliced_long['ECG'].unique())), max(list(sliced_long['ECG'].unique()))])),
    tooltip=['ECG'],
    color=alt.Color('channel:N', scale=channel_scale, title="ECG Channel")
).add_selection(
    channel_selector
).transform_filter(
    channel_selector
)

FullExtent = base.add_selection(
    interval
).properties(
    width=2000,
    height=100
)

ZoomedExtent = base.encode(
    x=alt.X('elapsedTime:Q', scale=alt.Scale(domain=interval.ref()), title="Time Elapsed (second)")
).properties(
    width=2000,
    height=500,
    title="Zoom and Channel Selection"
)

ZoomedExtent & FullExtent

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# make = pd.DataFrame({'channel': list(sliced_long['channel'].unique())})

# my_multi_selection = alt.selection_multi(fields=['channel'], name="channelselect")
# channel_scale = alt.Scale(domain=list(sliced_long['channel'].unique()),
#                          range=['#FF0000', '#00FF00', '#0099ff', '#4800ff'])

# interval = alt.selection_interval(encodings=['x'], name="intervalselect")

# color_condition = alt.condition(my_multi_selection, alt.Color('channel:N', scale=channel_scale), alt.value('lightgray'), legend=None)

# show_channel_selector = alt.Chart(make).mark_rect().encode(
#     y='channel', color=color_condition
# ).add_selection(my_multi_selection)

# base = alt.Chart(sliced_long).mark_line().encode(
#     x=alt.X('elapsedTime:Q', title="Time Elapsed (second)"),
#     y=alt.Y('ECG:Q', title="ECG Value", scale=alt.Scale(domain=[min(list(sliced_long['ECG'].unique())), max(list(sliced_long['ECG'].unique()))])),
#     tooltip=['ECG'],
#     color=alt.Color('channel:N', scale=channel_scale, title="ECG Channel")
# ).add_selection(
#     my_multi_selection
# ).transform_filter(
#     my_multi_selection
# )

# FullExtent = base.add_selection(
#     interval
# )

# ZoomedExtent = base.encode(
#     x=alt.X('elapsedTime:Q', scale=alt.Scale(domain=interval.ref()), title="Time Elapsed (second)")
# ).properties(
#     width=2000,
#     height=500,
#     title="Zoom and Channel Selection"
# )



# main_vis = ZoomedExtent & FullExtent

# final = show_channel_selector | main_vis

# final

In [ ]:
make = pd.DataFrame({'channel': list(sliced_long['channel'].unique())})

my_multi_selection = alt.selection_multi(fields=['channel'], name="channelselect")
channel_scale = alt.Scale(domain=list(sliced_long['channel'].unique()),
                         range=['#FF0000', '#00FF00', '#0099ff', '#4800ff'])

interval = alt.selection_interval(encodings=['x'], name="intervalselect")

color_condition = alt.condition(my_multi_selection, alt.Color('channel:N', scale=channel_scale), alt.value('lightgray'), legend=None)

show_channel_selector = alt.Chart(make).mark_rect().encode(
    y='channel', color=color_condition
).add_selection(my_multi_selection)

base = alt.Chart(sliced_long).mark_line().encode(
    x=alt.X('elapsedTime:Q', title="Time Elapsed (second)"),
    y=alt.Y('ECG:Q', title="ECG Value", scale=alt.Scale(domain=[min(list(sliced_long['ECG'].unique()))-10, max(list(sliced_long['ECG'].unique()))+10])),
    tooltip=['ECG'],
    color=alt.Color('channel:N', scale=channel_scale, title="ECG Channel")
).add_selection(
    my_multi_selection
).transform_filter(
    my_multi_selection
).properties(
    width=2000
)

FullExtent = base.add_selection(
    interval
).properties(
    # width=2000,
    height=200
)

ZoomedExtent = base.encode(
    x=alt.X('elapsedTime:Q', scale=alt.Scale(domain=interval.ref()), title="Time Elapsed (second)")
).properties(
    # width=2000,
    height=500,
    title="Zoom and Channel Selection"
)


main_vis = ZoomedExtent & FullExtent
final = main_vis & show_channel_selector
final

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
!unzip "/content/gdrive/My Drive/Research/edf_files.zip" 
# upload files manually works too

In [ ]:
%%capture
!pip install pyedflib
import pyedflib
from pyedflib import highlevel

signals1, signal1_headers1, header1 = highlevel.read_edf('/content/r01.edf')
signals4, signal_headers4, header4 = highlevel.read_edf('/content/r04.edf')
signals7, signal_headers7, header7 = highlevel.read_edf('/content/r07.edf')
signals8, signal_headers8, header8 = highlevel.read_edf('/content/r08.edf')
signals10, signal_headers10, header10 = highlevel.read_edf('/content/r10.edf')

In [ ]:
def columnRpeak(df, peaks):
  print("Amount of fetal R peaks:", len(peaks))
  print('Average fetal beat per minute:', len(peaks)/5)
  df['fetal R peaks'] = 0
  for i in range(len(peaks)):
    index = (peaks[i]*1000)
    df.at[index,'fetal R peaks']=1
  return df

In [ ]:
import pandas as pd

peaks1 = [row[0] for row in header1['annotations']] # Timestamps of fetal R peaks
r01 = pd.DataFrame({
    "direct":signals1[0],
    "ch1":signals1[1],
    "ch2":signals1[2],
    "ch3":signals1[3],
    "ch4":signals1[4],
})

r01 = columnRpeak(r01, peaks1)


In [ ]:
peaks4 = [row[0] for row in header4['annotations']]
r04 = pd.DataFrame({
    "direct":signals4[0],
    "ch1":signals4[1],
    "ch2":signals4[2],
    "ch3":signals4[3],
    "ch4":signals4[4]
})
r04 = columnRpeak(r04, peaks4)

In [ ]:
peaks7 = [row[0] for row in header7['annotations']] 
r07 = pd.DataFrame({
    "direct":signals7[0],
    "ch1":signals7[1],
    "ch2":signals7[2],
    "ch3":signals7[3],
    "ch4":signals7[4]
})
r07 = columnRpeak(r07, peaks7)

In [ ]:
peaks8 = [row[0] for row in header8['annotations']] 
r08 = pd.DataFrame({
    "direct":signals8[0],
    "ch1":signals8[1],
    "ch2":signals8[2],
    "ch3":signals8[3],
    "ch4":signals8[4]
})
r08 = columnRpeak(r08, peaks8)


In [ ]:
peaks10 = [row[0] for row in header10['annotations']]
r10 = pd.DataFrame({
    "direct":signals10[0],
    "ch1":signals10[1],
    "ch2":signals10[2],
    "ch3":signals10[3],
    "ch4":signals10[4]
})
r10 = columnRpeak(r10, peaks10)

In [ ]:
def intervalBPM(rpeaks):
  start = 0
  end = 7
  all_bpm = []
  all_peaks = []
  peaks = []
  for i in range(len(rpeaks)):
    if  rpeaks[i] >= start and rpeaks[i] <= end:
      peaks.append(rpeaks[i])
    else:
      bpm = round(len(peaks) * 60 / 7)
      all_bpm.append(bpm)
      all_peaks.append(peaks)
      if bpm < 110 or bpm > 160:
        print(peaks)
        print(bpm)
      peaks = []
      start += 7
      end += 7
      peaks.append(rpeaks[i])
  return all_bpm

In [ ]:
bpm1 = intervalBPM(peaks1)
bpm4 = intervalBPM(peaks4)
bpm7 = intervalBPM(peaks7)
bpm8 = intervalBPM(peaks8)
bpm10 = intervalBPM(peaks10)

# Generating Data
source = pd.DataFrame({
    'r01': bpm1,
    'r04': bpm4,
    'r07': bpm7,
    'r08': bpm8
    # 'r10': bpm10
})
df_long = source.melt(var_name="Mother", value_name="BPM")
alt.Chart(df_long).mark_area(
    opacity=0.5,
    interpolate='step'
).encode(
    alt.X('BPM:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('Mother:N')
).properties(
    title='Overlapping Histograms'
)